In [20]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import norm

import matplotlib.pyplot as plt

In [21]:
downweight_granularity=['dtime','Direction','dday','month']

# Data Import

In [22]:
Data=pd.read_csv('R_Output_Test.csv',sep=',')

In [23]:
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','NumPax']]

In [24]:
Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

In [25]:
Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

In [26]:
Data=Data.loc[Data['TicketDate']<=pd.to_datetime('2019-03-27'),:]

In [27]:
Data=Data[[x for x in Data.columns if x!='TicketDate']]

In [28]:
Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['NumPax'].cumsum().reset_index()

In [29]:
Data['Dprio']=-Data['Dprio']

In [30]:
IdealCurve=pd.read_csv('IdealCurve.csv')

In [31]:
Data=Data.merge(IdealCurve,on=downweight_granularity+['Dprio'])

In [32]:
Data=Data[['DepDate','dtime','Direction','dday','Dprio','NumPax','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]\
.sort_values(by=['DepDate','Direction','dtime','Dprio'])

In [33]:
Data['NumPax']=Data['NumPax'].apply(lambda x: int(round(x)))
Data['Ideal_lower']=Data['Ideal_lower'].apply(lambda x: int(round(x)))
Data['Ideal_average_downweighted']=Data['Ideal_average_downweighted'].apply(lambda x: int(round(x)))
Data['Ideal_average_full']=Data['Ideal_average_full'].apply(lambda x: int(round(x)))
Data['Ideal_upper']=Data['Ideal_upper'].apply(lambda x: int(round(x)))

In [34]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [35]:
Data=Data.sort_values(by=['DepDate','dtime'])
Data=Data.rename(columns={'NumPax': 'Actual Bookings', 'Ideal_lower': 'Ramp-up frontier',
'Ideal_average_downweighted': 'Ideal curve (80% LF)', 'Ideal_average_full': 'Ideal curve (100% LF)',
'Ideal_upper': 'Phase-down frontier'})

In [37]:
Data.to_csv('GUI_in.csv',index=False)

In [38]:
Data.loc[Data['DepDate']==pd.to_datetime('2019-04-01'),:]

,DepDate,dtime,Direction,dday,Dprio,Actual Bookings,Ramp-up frontier,Ideal curve (80% LF),Ideal curve (100% LF),Phase-down frontier
26766,2019-04-01,14:00,O,Monday,5,27,44,50,62,65
26767,2019-04-01,14:00,O,Monday,6,28,42,48,60,64
26768,2019-04-01,14:00,O,Monday,7,26,40,47,59,63
26769,2019-04-01,14:00,O,Monday,8,26,38,46,57,61
26770,2019-04-01,14:00,O,Monday,9,26,37,44,55,60
26771,2019-04-01,14:00,O,Monday,10,24,35,43,54,59
26772,2019-04-01,14:00,O,Monday,11,22,34,42,52,58
26773,2019-04-01,14:00,O,Monday,12,22,32,40,51,56
26774,2019-04-01,14:00,O,Monday,13,20,31,39,49,55
26775,2019-04-01,14:00,O,Monday,14,19,30,38,48,54
